In [87]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv
import os
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains


# from chromedriver_py import binary_path # this will get you the path variable

In [88]:
svc = webdriver.ChromeService(executable_path=r"C:\chromedriver.exe")
options = Options()
options.add_argument("start-maximized"); #// open Browser in maximized mode
options.add_argument("disable-infobars"); #// disabling infobars
options.add_argument("--disable-extensions"); #// disabling extensions
options.add_argument("--disable-gpu"); #// applicable to windows os only
options.add_argument("--disable-dev-shm-usage"); #// overcome limited resource problems
options.add_argument("--no-sandbox"); #// Bypass OS security model
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"   # Do not wait for full page

In [89]:
user_agent = {"User-Agent":"risk_takers"}
amz_home = "https://www.amazon.in/"

In [90]:
# Open Amazon india home page
browser = webdriver.Chrome(service=svc, options=options)
browser.get(amz_home)

In [91]:
# Operations to be done on single page(view)

# click on Navigation menu
nav_menu = browser.find_elements(By.ID, "nav-hamburger-menu")
nav_menu[0].click()

# scroll to 'Mobiles, Computers' Section under 'Shop By Category'
# cat_option = browser.find_element_by_xpath('//a[@href="'+url+'"]') # find_element_by_xpath() is depricated
# cat_option = browser.find_elements(By.XPATH, '//a[@data-ref-tag="'+"nav_em_1_1_1_14"+'"]')
# cat_option = browser.find_elements(By.LINK_TEXT, "Mobiles, Computers")
# cat_option = browser.find_elements(By.LINK_TEXT, "Shop By Category")
# browser.execute_script("arguments[0].scrollIntoView();", cat_option[0])
# print(cat_option)
# print(len(cat_option))
# cat_option[0].location_once_scrolled_into_view

browser.execute_script("arguments[0].click();", 
                      WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "Mobiles, Computers"))))


# click on the first Category, in this case 'Mobiles, Computers' is the first category, hence not searching again
# cat_option[0].click()

# Search for element 'All Mobile Phones'
# Note: take href from browser address. It should NOT contain &amp; or anyother text separator
# Wrong href: /gp/browse.html?node=1389401031&amp;ref_=nav_em_sbc_mobcomp_all_mobiles_0_2_8_2
# Correct href: /gp/browse.html?node=1389401031&ref_=nav_em_sbc_mobcomp_all_mobiles_0_2_8_2
# sub_cat = browser.find_elements(By.XPATH, '//a[@href="'+\
#                                 "/gp/browse.html?node=1389401031&ref_=nav_em_sbc_mobcomp_all_mobiles_0_2_8_2"+\
#                                 '"]')
browser.execute_script("arguments[0].click();", 
                      WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//a[@href="'+\
                                "/gp/browse.html?node=1389401031&ref_=nav_em_sbc_mobcomp_all_mobiles_0_2_8_2"+\
                                '"]'))))
# sub_cat[0].click()


# ActionChains(browser)\
#         .scroll_to_element(cat_option[0])\
#         .perform()


In [92]:
# Apply some filters to only show smart phones
smart_basic_phones = browser.find_elements(By.LINK_TEXT, "Smartphones & Basic Mobiles")
smart_basic_phones[0].click()

smart_phones = browser.find_elements(By.LINK_TEXT, "Smartphones")
smart_phones[0].click()

In [93]:
# Lets filter some phone specific to a brand

# First scroll to 'Storage max' Filter
brand_filter_label_elem = browser.find_elements(By.LINK_TEXT, "512 GB & above")
browser.execute_script("arguments[0].scrollIntoView();", brand_filter_label_elem[0])

brand_filter_href_apple = "Apple"
# "apb-browse-refinements-checkbox_7"
# "/s?bbn=1389401031&rh=n%3A1389401031%2Cp_89%3AApple&dc&qid=1702737138&rnid=3837712031&ref=lp_1389401031_nr_p_89_3"
# "/s?bbn=1389401031&rh=n%3A1389401031%2Cp_89%3AApple&dc&qid=1702741971&rnid=3837712031&ref=lp_1389401031_nr_p_89_2"
brand_filter_href_samsung = "Samsung"
brand_filter_href_oneplus = "OnePlus"
brand_filter_href_xiaomi = "Xiaomi"

brands = ["Apple", "Samsung", "OnePlus", "Xiaomi", "realme", "Redmi", "iQOO"]
# apple_filter_button_elem = browser.find_elements(By.XPATH, '//a[@href="'+brand_filter_href_apple+'"]')
# apple_filter_button_elem = browser.find_elements(By.XPATH, '//span[contains(text(), "Apple")]')
# apple_filter_button_elem = browser.find_elements(By.XPATH, '//a[@aria-labelledby="'+"Apple"+'"]')

# apple_filter_button_elem = browser.find_elements(By.LINK_TEXT, brand_filter_href_apple)
# apple_filter_button_elem[0].click()

# print(browser.current_url)

In [94]:
# The "See all results" route
product_name = []
mrp = []
selling_price = []
ratings = []
counter = 1
bought_last_month = []
offer_percentage = []
clear_brand_filter = "Clear"

# for brand in brands:
# print("Brand: ", brand)
# brand_elem = browser.find_elements(By.LINK_TEXT, brand)
brand_elem = browser.find_elements(By.LINK_TEXT, "See all results")
brand_elem[0].click()

# curr_filter_link = browser.current_url

#logic to get products
# Use Beautifulsoup to download the html structure of the Amazon Page
soup = BeautifulSoup(browser.page_source, features="html.parser") # we need a parser to parse the content
#     print(soup)

while(True):
    # searching the Parent tag...
    products =  soup.find_all("div", class_="sg-col-inner")
    print("Potential Products:", len(products))
    # print(products[6])

    # Finding the Product Name    
    for p in products:
        # Finding the Product Name
        if(p.find("span", class_="a-size-base-plus a-color-base a-text-normal")):
            product_name.append(p.find("span", class_="a-size-base-plus a-color-base a-text-normal").text.strip().split(",")[0])
        else:
            # if product name is not found, continue to next element
            continue

        # Finding the Selling Price
        if p.find("span", class_="a-price"):
            selling_price.append(p.find("span", class_="a-offscreen").text)
        else:
            selling_price.append(np.nan)

        # Finding the MRP
        if p.find("span", class_="a-price a-text-price"):
            mrp.append(p.find("span", class_="a-price a-text-price").text.strip().split("₹")[1])
        else:
            mrp.append(np.nan)

        # Finding the Ratings
        if p.find("div", class_="a-row a-size-small"):
            ratings.append(p.find("div", class_="a-row a-size-small").text.strip())
        else:
            ratings.append(np.nan)

        # Finding the Items Boughts Last Month
        if p.find("div", class_="a-row a-size-base"):
            bought_last_month.append(p.find("div", class_="a-row a-size-base").text)
        else:
            bought_last_month.append(np.nan)

        #Get % off during feature engineering phase
        # (mrp-selling_price)/mrp*100
    print("Page: ", counter)
    counter += 1
        
    # if(len(browser.find_elements(By.LINK_TEXT, "Next"))):
    #     browser.execute_script("arguments[0].click();", 
    #                     WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "Next"))))
    # else:
    #     break
    pagination_next = soup.find_all('a', {'class': 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'})
    if not any(a.get_text() == 'Next' or a.has_attr("aria-disabled") for a in pagination_next):
        break  # Stop scraping if there are no more pages
    else:
        browser.execute_script("arguments[0].click();", 
                        WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "Next"))))

print("Total Identified Products:", len(product_name))

# browser.get(curr_filter_link)

# #clear filter
# clear_filter_button_elem = browser.find_elements(By.LINK_TEXT, clear_brand_filter)
# clear_filter_button_elem[0].click()

Potential Products: 31
Page:  1
Potential Products: 31
Page:  2
Potential Products: 31
Page:  3
Potential Products: 31
Page:  4
Potential Products: 31
Page:  5
Potential Products: 31
Page:  6


In [54]:
# product_name = []
# mrp = []
# selling_price = []
# ratings = []
# counter = 1
# bought_last_month = []
# offer_percentage = []
# clear_brand_filter = "Clear"

# for brand in brands:
#     print("Brand: ", brand)
#     brand_elem = browser.find_elements(By.LINK_TEXT, brand)
#     brand_elem[0].click()
    
#     curr_filter_link = browser.current_url
    
#     #logic to get products
#     # Use Beautifulsoup to download the html structure of the Amazon Page
#     soup = BeautifulSoup(browser.page_source, features="html.parser") # we need a parser to parse the content
# #     print(soup)
    
#     while(True):
#         # searching the Parent tag...
#         products =  soup.find_all("div", class_="sg-col-inner")
#         print("Potential Products:", len(products))
#         # print(products[6])

#         # Finding the Product Name    
#         for p in products:
#             # Finding the Product Name
#             if(p.find("span", class_="a-size-base-plus a-color-base a-text-normal")):
#                 product_name.append(p.find("span", class_="a-size-base-plus a-color-base a-text-normal").text.strip().split(",")[0])
#             else:
#                 # if product name is not found, continue to next element
#                 continue

#             # Finding the Selling Price
#             if p.find("span", class_="a-price"):
#                 selling_price.append(p.find("span", class_="a-offscreen").text)
#             else:
#                 selling_price.append(np.nan)

#             # Finding the MRP
#             if p.find("span", class_="a-price a-text-price"):
#                 mrp.append(p.find("span", class_="a-price a-text-price").text.strip().split("₹")[1])
#             else:
#                 mrp.append(np.nan)

#             # Finding the Ratings
#             if p.find("div", class_="a-row a-size-small"):
#                 ratings.append(p.find("div", class_="a-row a-size-small").text.strip())
#             else:
#                 ratings.append(np.nan)

#             # Finding the Items Boughts Last Month
#             if p.find("div", class_="a-row a-size-base"):
#                 bought_last_month.append(p.find("div", class_="a-row a-size-base").text)
#             else:
#                 bought_last_month.append(np.nan)

#             #Get % off during feature engineering phase
#             # (mrp-selling_price)/mrp*100
#         print("Page: ", counter)
#         counter += 1
            
#         if(len(browser.find_elements(By.LINK_TEXT, "Next"))):
#             browser.execute_script("arguments[0].click();", 
#                             WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "Next"))))
#         else:
#             break

#     print("Total Identified Products:", len(product_name))
    
#     browser.get(curr_filter_link)
    
#     #clear filter
#     clear_filter_button_elem = browser.find_elements(By.LINK_TEXT, clear_brand_filter)
#     clear_filter_button_elem[0].click()

Brand:  Apple
Potential Products: 31
Page:  1
Potential Products: 31
Page:  2
Potential Products: 31
Page:  3
Total Identified Products: 75
Brand:  Samsung


IndexError: list index out of range

In [86]:
# Store the Following Information in a DataFrame
import pandas as pd
prod_info = pd.DataFrame(product_name, columns = ["Product_Name"])
prod_info["Selling Price"] = pd.Series(selling_price)
prod_info["MRP"] = pd.Series(mrp)
# prod_info["% Off"] = pd.Series(offer_percentage)
prod_info["Items Bought Last Month"] = pd.Series(bought_last_month)
prod_info["Ratings"] = pd.Series(ratings)

# Generating a csv file.
prod_info.to_csv("Prod_Info.csv", mode='a', index = False)

# prod_info.isnull().sum()
prod_info

,Product_Name,Selling Price,MRP,Items Bought Last Month,Ratings
0,Infinix HOT 30i (Glacier Blue,"₹9,399","11,999",1K+ bought in past month,4.0 out of 5 stars 1
1,Infinix HOT 30i (Glacier Blue,"₹9,399","11,999",NaN,4.0 out of 5 stars 1
2,Infinix HOT 30i (Glacier Blue,"₹9,799","11,999",NaN,3.2 out of 5 stars 20
3,G42 5G | Snapdragon® 480+ 5G | 50MP Triple AI ...,"₹12,599","16,499",1K+ bought in past month,"3.8 out of 5 stars 1,852"
4,narzo 60X 5G（Nebula Purple 6GB,"₹14,499","15,999",5K+ bought in past month,"4.1 out of 5 stars 5,830"
...,...,...,...,...,...
5007,narzo 50i Prime (Mint Green 4GB RAM+64GB Stora...,"₹6,399","9,999",5K+ bought in past month,"4.0 out of 5 stars 8,186"
5008,13C 5G (Startrail Green,"₹12,499","15,999",5K+ bought in past month,4.1 out of 5 stars 112
5009,Z7s 5G by vivo (Norway Blue,"₹16,999","24,999",5K+ bought in past month,"4.1 out of 5 stars 12,600"
5010,narzo N55 (Prime Black,"₹9,499","14,999",5K+ bought in past month,"4.1 out of 5 stars 10,508"


In [87]:
#clear filter
clear_filter_button_elem = browser.find_elements(By.LINK_TEXT, clear_brand_filter)
clear_filter_button_elem[0].click()